In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_features.head()

In [ ]:
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
test_features.head()

In [ ]:
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets.head()

In [ ]:
np.unique(train_features['cp_type'])

In [ ]:
np.unique(train_features['cp_time'])

In [ ]:
np.unique(train_features['cp_dose'])

In [ ]:
cp_type = np.array(train_features['cp_type'])
print('Train')
print('count of trt_cp:', (cp_type == 'trt_cp').sum())
print('count of ctl_vehicle:', (cp_type == 'ctl_vehicle').sum())
cp_type = np.array(test_features['cp_type'])
print('Test')
print('count of trt_cp:', (cp_type == 'trt_cp').sum())
print('count of ctl_vehicle:', (cp_type == 'ctl_vehicle').sum())

Classes are disbalanced

In [ ]:
cp_time = np.array(train_features['cp_time'])
print('Train')
print('count of 24:', (cp_time == 24).sum())
print('count of 48:', (cp_time == 48).sum())
print('count of 72:', (cp_time == 72).sum())
cp_time = np.array(test_features['cp_time'])
print('Test')
print('count of 24:', (cp_time == 24).sum())
print('count of 48:', (cp_time == 48).sum())
print('count of 72:', (cp_time == 72).sum())

In [ ]:
cp_dose = np.array(train_features['cp_dose'])
print('Train')
print('count of D1:', (cp_dose == 'D1').sum())
print('count of D2:', (cp_dose == 'D2').sum())
cp_dose = np.array(test_features['cp_dose'])
print('Test')
print('count of D1:', (cp_dose == 'D1').sum())
print('count of D2:', (cp_dose == 'D2').sum())

In [ ]:
train =  train_features.merge(train_targets, on = 'sig_id')

In [ ]:
train.shape

In [ ]:
d = ['cp_type']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[train1['cp_type'] == 'ctl_vehicle'])[:, 1:]
(ar != 0).sum()

### !!! All targets on (cp_type = ctl_vehicle) == 0

In [ ]:
d = ['cp_type']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[train1['cp_type'] == 'trt_cp'])[:, 1:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

A lot of zero values

Continue research

In [ ]:
d = ['cp_type', 'cp_time']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp') & (train1['cp_time'] == 24)])[:, 2:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
d = ['cp_type', 'cp_time']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp') & (train1['cp_time'] == 48)])[:, 2:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
d = ['cp_type', 'cp_time']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp') & (train1['cp_time'] == 72)])[:, 2:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
d = ['cp_type', 'cp_dose']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp') & (train1['cp_dose'] == 'D1')])[:, 2:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
d = ['cp_type', 'cp_dose']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp') & (train1['cp_dose'] == 'D2')])[:, 2:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
d = ['cp_type', 'cp_time', 'cp_dose']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp') & (train1['cp_time'] == 72) & (train1['cp_dose'] == 'D2')])[:, 3:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In all parts of data percent of non-zero targets is about the same

Look at histograms of random features

In [ ]:
plt.figure(figsize = (20, 8))
plt.subplot(241)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(242)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(243)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(244)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(245)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(246)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(247)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.subplot(248)
n = np.random.randint(772)
plt.hist(train_features['g-' + str(n)], bins = 70)
plt.title('Histogram(' + 'g-' + str(n) + ')', fontsize = 16)
plt.show()

In [ ]:
plt.figure(figsize = (20, 8))
plt.subplot(241)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(242)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(243)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(244)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(245)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(246)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(247)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.subplot(248)
n = np.random.randint(100)
plt.hist(train_features['c-' + str(n)], bins = 70)
plt.title('Histogram(' + 'c-' + str(n) + ')', fontsize = 16)
plt.show()

Look at scatter of random features and random targets

In [ ]:
plt.figure(figsize = (25, 8))
plt.subplot(241)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(242)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(243)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(244)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(245)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(246)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(247)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(248)
n = np.random.randint(100)
m = np.random.randint(1, 207)
plt.scatter(train_features['c-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'c-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.show()

In [ ]:
plt.figure(figsize = (25, 8))
plt.subplot(241)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(242)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(243)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(244)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(245)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(246)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(247)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.subplot(248)
n = np.random.randint(772)
m = np.random.randint(1, 207)
plt.scatter(train_features['g-' + str(n)], train_targets[train_targets.columns[m]])
plt.title('Scatter(' + 'g-' + str(n) + ' - ' + train_targets.columns[m] +')', fontsize = 12)
plt.show()

In [ ]:
d = ['cp_type', 'cp_time']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp')])[:, 2:]
(ar != 0).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
d = ['cp_type', 'cp_time']
d.extend(train_targets.columns[1:])
train1 = train[d]
ar = np.array(train1[(train1['cp_type'] == 'trt_cp')])[:, 2:]
(ar != 1).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
(ar == 0).sum() / (ar.shape[0] * ar.shape[1]) + (ar == 1).sum() / (ar.shape[0] * ar.shape[1])

In [ ]:
ar.shape

Print percent of values 0 and 1 for each feature

In [ ]:
n = train_targets.shape[0]
for x in train_targets.columns[1:]:
    ar1 = np.array(train_targets[x])
    print(x + ':' + ' ' * (50 - len(x)), np.round((ar1 == 0).sum() / n, 5), np.round((ar1 == 1).sum() / n, 5))

### Add additional features

https://www.kaggle.com/kushal1506/moa-pytorch-feature-engineering-0-01846

In [ ]:
features_g = ['g-' + str(i) for i in range(772)]
features_c = ['c-' + str(i) for i in range(100)]

In [ ]:
for df in train_features, test_features:
    df['g_sum'] = df[features_g].sum(axis = 1)
    df['g_mean'] = df[features_g].mean(axis = 1)
    df['g_median'] = df[features_g].median(axis = 1)
    df['g_std'] = df[features_g].std(axis = 1)
    df['g_kurt'] = df[features_g].kurtosis(axis = 1)
    df['g_skew'] = df[features_g].skew(axis = 1)
    df['c_sum'] = df[features_c].sum(axis = 1)
    df['c_mean'] = df[features_c].mean(axis = 1)
    df['c_std'] = df[features_c].std(axis = 1)
    df['c_median'] = df[features_c].median(axis = 1)
    df['c_kurt'] = df[features_c].kurtosis(axis = 1)
    df['c_skew'] = df[features_c].skew(axis = 1)
    df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
    df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
    df['gc_std'] = df[features_g + features_c].std(axis = 1)
    df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
    df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
    df['gc_median'] = df[features_g + features_c].median(axis = 1)
        

## Model

A lot of code is from kaggle.com/nicohrubec/pytorch-multilabel-neural-network/

In [ ]:
train_targets = train_targets[train_features['cp_type'] == 'trt_cp']
train_features = train_features[train_features['cp_type'] == 'trt_cp']
train_features = train_features.drop(columns = ['sig_id', 'cp_type'])
test1_features = test_features.copy()
test_features = test_features.drop(columns = ['sig_id', 'cp_type'])
cat_columns = ['cp_time', 'cp_dose']
cat_train_features = train_features[cat_columns]
cat_test_features = test_features[cat_columns]
train_features = train_features.drop(columns = cat_columns)
test_features = test_features.drop(columns = cat_columns)
train = np.array(train_features)
test = np.array(test_features)
targets = train_targets.columns
train_targets = np.array(train_targets[train_targets.columns[1:]])

In [ ]:
batch_size = 128
val_batch_size = batch_size * 4

In [ ]:
one_hot = OneHotEncoder()
X2_train = one_hot.fit_transform(cat_train_features)
X2_test = one_hot.transform(cat_test_features)

In [ ]:
train = np.concatenate((X2_train.toarray(), train), 1)
test = np.concatenate((X2_test.toarray(), test), 1)

In [ ]:
train.shape

In [ ]:
n_folds = 7
n_starts = 1
n_epochs = 70
n_targets = train_targets.shape[1]

In [ ]:
criterion = nn.BCELoss()
kfold = MultilabelStratifiedKFold(n_splits = 7, random_state = 42, shuffle = True)

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self, num_columns):
        super(SimpleNet, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_columns)
        self.dropout1 = nn.Dropout(0.4)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_columns, 1024))
        
        self.batch_norm2 = nn.BatchNorm1d(1024)
        self.dropout2 = nn.Dropout(0.4)
        self.dense2 = nn.utils.weight_norm(nn.Linear(1024, 1024))
        
        self.batch_norm3 = nn.BatchNorm1d(1024)
        self.dropout3 = nn.Dropout(0.4)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1024, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.sigmoid(self.dense3(x))
        
        return x

In [ ]:
class Dataset1(Dataset):
    def __init__(self, df, targets, mode='train'):
        self.mode = mode
        self.df = df
        if mode == 'train':
            self.targets = targets
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        if self.mode == 'train':
            return torch.FloatTensor(self.df[index]), torch.FloatTensor(self.targets[index])
        elif self.mode == 'test':
            return torch.FloatTensor(self.df[index]), 0

In [ ]:
if torch.cuda.is_available():
    device = 'cuda'
device

In [ ]:
submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
submission.columns

In [ ]:
train.shape

In [ ]:
train_targets.shape

In [ ]:
for n, (tr, te) in enumerate(kfold.split(train_targets, train_targets)):
    print('Train fold:', n + 1)
    xtrain, xval = train[tr], train[te]
    ytrain, yval = train_targets[tr], train_targets[te]

    train_set = Dataset1(xtrain, ytrain)
    val_set = Dataset1(xval, yval)

    dataloaders = {
        'train': DataLoader(train_set, batch_size=batch_size, shuffle=True),
        'val': DataLoader(val_set, batch_size=val_batch_size, shuffle=False)
    }

    model = SimpleNet(895).to(device)
    checkpoint_path = f'repeat:{0}_Fold:{n+1}.pt'
    optimizer = optim.Adam(model.parameters(), weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, eps=1e-4, verbose=True)
    best_loss = {'train': np.inf, 'val': np.inf}

    for epoch in range(n_epochs):
        epoch_loss = {'train': 0.0, 'val': 0.0}

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0

            for i, (x, y) in enumerate(dataloaders[phase]):
                x, y = x.to(device), y.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase=='train'):
                    preds = model(x)
                    loss = criterion(preds, y)

                    if phase=='train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() / len(dataloaders[phase])

            epoch_loss[phase] = running_loss

        print("Epoch {}/{}   -   loss: {:5.5f}   -   val_loss: {:5.5f}".format(epoch + 1, n_epochs, epoch_loss['train'], epoch_loss['val']))

        scheduler.step(epoch_loss['val'])

        if epoch_loss['val'] < best_loss['val']:
            best_loss = epoch_loss
            torch.save(model.state_dict(), checkpoint_path)

In [ ]:
oof = np.zeros((train.shape[0], n_targets - 1))
oof_targets = np.zeros((train.shape[0], n_targets - 1))
preds = np.zeros((test.shape[0], n_targets - 1))

In [ ]:
def mean_log_loss(y_true, y_pred):
    metrics = []
    for i, target in enumerate(targets[1:]):
        metrics.append(log_loss(y_true[:, i], y_pred[:, i].astype(float), labels=[0,1]))
    return np.mean(metrics)

In [ ]:
test.shape

In [ ]:
seed_targets = []
seed_oof = []
seed_preds = np.zeros((test.shape[0], n_targets, n_folds))

for n, (tr, te) in enumerate(kfold.split(train_targets, train_targets)):
    xval, yval = train[te], train_targets[te]
    fold_preds = []

    val_set = Dataset1(xval, yval)
    test_set = Dataset1(test, None, mode='test')

    dataloaders = {
        'val': DataLoader(val_set, batch_size=val_batch_size, shuffle=False),
        'test': DataLoader(test_set, batch_size=val_batch_size, shuffle=False)
    }

    checkpoint_path = f'repeat:{0}_Fold:{n+1}.pt'
    model = SimpleNet(895).to(device)
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()

    for phase in ['val', 'test']:
        for i, (x, y) in enumerate(dataloaders[phase]):
            if phase == 'val':
                x, y = x.to(device), y.to(device)
            elif phase == 'test':
                x = x.to(device)

            with torch.no_grad():
                batch_preds = model(x)

                if phase == 'val':
                    seed_targets.append(y)
                    seed_oof.append(batch_preds)
                elif phase == 'test':
                    fold_preds.append(batch_preds)

    fold_preds = torch.cat(fold_preds, dim=0).cpu().numpy()
    seed_preds[:, :, n] = fold_preds

seed_targets = torch.cat(seed_targets, dim=0).cpu().numpy()
seed_oof = torch.cat(seed_oof, dim=0).cpu().numpy()
seed_preds = np.mean(seed_preds, axis=2)

print("Overall score is {:5.5f}".format(mean_log_loss(seed_targets, seed_oof)))

In [ ]:
submission[targets[1:]] = seed_preds
submission.loc[test1_features['cp_type']=='ctl_vehicle', targets[1:]] = 0
submission.to_csv('submission.csv', index=False)